<a href="https://colab.research.google.com/github/MIlanJP/Facial_emotion_detection__/blob/main/Facial_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Required Libraries


In [2]:
# Importing libraries and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import tensorflow.keras.layers as tfl
import os 
from torch.utils.data import DataLoader
import torchvision as tv
from torchvision import transforms 

In [3]:
from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# Downloading the dataset
!gdown --id 1kdAI0wrR_bTBs-6p/bKB6giz-JcNaTY-s

# Unzipping the files
!unzip '/content/drive/MyDrive/archive.zip'

In [6]:
# Defining te path of train and test dataset
train_dir = '/content/train'
test_dir = '/content/test'

# Data Augumentation and  Performing train test split 
## NOTE keeping the dimensions of the image 48 X 48 pixels (Grayscale)
- Train Split
- Test Split

In [8]:
# Generating batches of tensor image data with real-time data augmentation.
batch = 64
img = (48,48)
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20, rescale=1./255, shear_range=0.1,
                                                            zoom_range=0.2, horizontal_flip=True, vertical_flip=True,
                                                            width_shift_range=0.1,  height_shift_range=0.1, 
                                                            brightness_range=(0.2, 0.8))

train_data = train_gen.flow_from_directory(train_dir, target_size=img, batch_size=batch, class_mode = "categorical",
                                           color_mode="grayscale"
                                           )

validation_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_data = validation_gen.flow_from_directory(test_dir, target_size=img, batch_size=batch*2, class_mode='categorical',
                                                     color_mode="grayscale"
                                                     )

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
